In [ ]:
# ============================================================
# Environment Setup
# ============================================================

import sys
import os
import warnings
warnings.filterwarnings('ignore')

project_root = os.path.abspath('../..')
if project_root not in sys.path:
    sys.path.append(project_root)

import sagemaker
import boto3
from sagemaker import get_execution_role
from sagemaker.sklearn import SKLearnModel
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import CSVDeserializer
import pandas as pd
import numpy as np
import time
import json
from datetime import datetime, timedelta

try:
    from utils.sagemaker_config import get_sagemaker_config
    config = get_sagemaker_config(s3_prefix='lab8-deployment-strategies')
    role = config['role']
    session = config['session']
    bucket = config['bucket']
    region = config['region']
except ImportError:
    role = get_execution_role()
    session = sagemaker.Session()
    bucket = session.default_bucket()
    region = session.boto_region_name

print(f"Configuration complete")
print(f"Region: {region}")
print(f"S3 Bucket: s3://{bucket}")

sm_client = boto3.client('sagemaker', region_name=region)
sm_runtime = boto3.client('sagemaker-runtime', region_name=region)
cloudwatch = boto3.client('cloudwatch', region_name=region)

---

## Section 1: Préparer Deux Versions de Modèle


---

## Section 1: SageMaker Projects - MLOps Template for Fraud Detection

**SageMaker Projects** provides MLOps templates with built-in CI/CD pipelines for production ML workflows.

### What is a SageMaker Project?

SageMaker Projects are pre-configured templates that include:
- **Source Code Repository** (CodeCommit or GitHub)
- **CI/CD Pipeline** (CodePipeline + CodeBuild)
- **Model Registry Integration**
- **Automated Deployment** (Dev, Staging, Production)
- **Infrastructure as Code** (CloudFormation)

### Benefits for Fraud Detection

| Feature | Benefit |
|---------|---------|
| Model Versioning | Track all fraud model versions |
| Automated Training | Retrain when new fraud patterns detected |
| Multi-Environment | Test in staging before production |
| Approval Workflow | Manual approval for fraud model changes |
| Audit Trail | Complete history of model deployments |

### Project Templates Available

1. **MLOps template for model building, training, and deployment**
   - End-to-end CI/CD for fraud detection
   - Automated retraining pipeline
   - Multi-account deployment

2. **MLOps template for model deployment**
   - Focus on deployment strategies
   - Blue/Green and Canary deployments
   - Rollback capabilities

3. **Custom templates**
   - Organization-specific workflows
   - Compliance requirements

In [ ]:
# ============================================================
# Create SageMaker Project for Fraud Detection
# ============================================================

print("SageMaker Projects - Fraud Detection MLOps Pipeline")
print("=" * 60)

# Note: Projects require SageMaker Studio and appropriate IAM permissions
# This demonstrates the API; actual creation is best done via Console first

project_name = "fraud-detection-mlops"

# Example: Create project using built-in template
project_template_example = """
# Via AWS Console:
# 1. Open SageMaker Studio
# 2. Go to Projects → Create Project
# 3. Select: "MLOps template for model building, training, and deployment"
# 4. Project name: fraud-detection-mlops
# 5. This creates:
#    - CodeCommit repo with pipeline code
#    - SageMaker Pipeline for training
#    - CodePipeline for deployment
#    - Model Registry integration
#    - Dev and Prod endpoints

# Via CLI:
aws sagemaker create-project \\
    --project-name fraud-detection-mlops \\
    --service-catalog-provisioning-details '{
        "ProductId": "prod-xxxxxx",
        "ProvisioningArtifactId": "pa-xxxxxx"
    }' \\
    --tags Key=UseCase,Value=FraudDetection
"""

print(project_template_example)

# List available project templates
sm_client = boto3.client('sagemaker')

print("\n\n📋 Checking SageMaker Project Templates:")
print("-" * 60)

try:
    # List service catalog products (project templates)
    sc_client = boto3.client('servicecatalog')
    
    # Note: This requires appropriate permissions
    print("Available templates (requires SageMaker Studio and IAM permissions):")
    print("1. MLOps template for model building, training, and deployment")
    print("2. MLOps template for model deployment")
    print("3. Custom organizational templates")
    
except Exception as e:
    print(f"Note: Service Catalog access requires SageMaker Studio setup")
    print(f"Typically configured through SageMaker Studio UI")

print("\n\n🏗️  Project Structure (once created):")
print("-" * 60)
print("""
fraud-detection-mlops/
├── 📁 CodeCommit Repository
│   ├── pipelines/
│   │   ├── fraud/
│   │   │   ├── pipeline.py          # Training pipeline definition
│   │   │   ├── preprocess.py        # Data preprocessing
│   │   │   ├── evaluate.py          # Model evaluation
│   │   │   └── requirements.txt
│   │   └── codebuild-buildspec.yml  # CI/CD build spec
│   ├── tests/                       # Unit tests
│   └── README.md
│
├── 🔄 CodePipeline (Build)
│   ├── Source (CodeCommit trigger)
│   ├── Build (CodeBuild)
│   └── Deploy Pipeline to SageMaker
│
├── 🚀 CodePipeline (Deploy)
│   ├── Source (Model Registry approval)
│   ├── DeployStaging (Test environment)
│   ├── Manual Approval
│   └── DeployProd (Production)
│
└── 📊 Model Registry
    └── fraud-detection-model-group
        ├── Version 1 (Approved)
        ├── Version 2 (PendingApproval)
        └── Version 3 (Draft)
""")

In [ ]:
# ============================================================
# Explore SageMaker Pipeline (Created by Project)
# ============================================================

print("SageMaker Pipeline - Automated Training Workflow")
print("=" * 60)

# Once project is created, it generates a SageMaker Pipeline
# Let's show what a fraud detection pipeline would contain

pipeline_structure = """
Pipeline Name: fraud-detection-pipeline

Steps:
1. 📥 Data Processing
   - Input: Raw fraud transaction data from S3
   - Process: Feature engineering, train/test split
   - Output: Processed data for training

2. 🏋️  Model Training
   - Algorithm: XGBoost with SMOTE
   - Hyperparameters: Auto-tuned or configured
   - Output: Trained model artifact

3. 📊 Model Evaluation
   - Metrics: Precision, Recall, F1, ROC-AUC
   - Threshold: Minimum performance criteria
   - Decision: Register model if metrics pass

4. 🏷️  Conditional Model Registration
   - Condition: If evaluation passes threshold
   - Action: Register to Model Registry
   - Status: PendingManualApproval

5. 🚀 Trigger Deployment Pipeline
   - Event: Model approval in registry
   - Action: Start deployment to staging
"""

print(pipeline_structure)

# Example: Query existing pipelines
print("\n\n🔍 Checking Existing Pipelines:")
print("-" * 60)

try:
    pipelines = sm_client.list_pipelines(
        MaxResults=10
    )
    
    if pipelines['PipelineSummaries']:
        print(f"Found {len(pipelines['PipelineSummaries'])} pipelines:")
        for pipeline in pipelines['PipelineSummaries']:
            print(f"  • {pipeline['PipelineName']}")
            print(f"    Created: {pipeline['CreationTime']}")
    else:
        print("No pipelines found yet.")
        print("Pipelines are automatically created when you create a SageMaker Project.")
        
except Exception as e:
    print(f"Note: {e}")

print("\n\n💡 Pipeline Benefits for Fraud Detection:")
print("-" * 60)
print("""
✓ Automated Retraining
  - Schedule: Daily/weekly checks for data drift
  - Trigger: New fraud patterns detected
  - Action: Automatic pipeline execution

✓ Consistent Training Process
  - Same steps every time
  - Version-controlled code
  - Reproducible results

✓ Built-in Quality Gates
  - Minimum AUC threshold (e.g., 0.85)
  - Precision/recall requirements
  - Only deploy models that pass

✓ CI/CD Integration
  - Code change → Pipeline update
  - Model approval → Auto-deployment
  - Rollback capability
""")

In [ ]:
# ============================================================
# Model Registry Integration with Projects
# ============================================================

print("Model Registry - Approval Workflow for Fraud Models")
print("=" * 60)

# SageMaker Projects integrate with Model Registry for approval workflow

# Example: List model packages (versions) in registry
model_package_group_name = "fraud-detection-model-group"

print(f"\n📦 Model Package Group: {model_package_group_name}")
print("-" * 60)

try:
    # Check if model package group exists
    response = sm_client.list_model_package_groups(
        NameContains='fraud'
    )
    
    if response['ModelPackageGroupSummaryList']:
        print(f"Found {len(response['ModelPackageGroupSummaryList'])} fraud-related model groups:")
        for group in response['ModelPackageGroupSummaryList']:
            print(f"\n  📁 {group['ModelPackageGroupName']}")
            print(f"     Created: {group['CreationTime']}")
            
            # List versions in this group
            versions = sm_client.list_model_packages(
                ModelPackageGroupName=group['ModelPackageGroupName'],
                MaxResults=5,
                SortBy='CreationTime',
                SortOrder='Descending'
            )
            
            if versions['ModelPackageSummaryList']:
                print(f"     Versions: {len(versions['ModelPackageSummaryList'])}")
                for version in versions['ModelPackageSummaryList']:
                    status = version['ModelApprovalStatus']
                    status_icon = "✅" if status == "Approved" else "⏳" if status == "PendingManualApproval" else "❌"
                    print(f"       {status_icon} v{version['ModelPackageVersion']}: {status}")
    else:
        print("No fraud model groups found yet.")
        print("\n💡 Model Registry is automatically created with SageMaker Projects")
        
except Exception as e:
    print(f"Info: {e}")
    print("\n💡 Model Registry Setup:")

print("\n\n🔄 Approval Workflow in Projects:")
print("-" * 60)
print("""
1. Pipeline Training Completes
   ├─ Model artifacts saved to S3
   ├─ Evaluation metrics calculated
   └─ IF metrics pass → Register model

2. Model Registered (PendingManualApproval)
   ├─ Notification sent to ML team
   ├─ Review metrics dashboard
   └─ Manual review required

3. Data Scientist Approves Model
   ├─ Change status: PendingManualApproval → Approved
   └─ Triggers deployment pipeline automatically

4. Deployment Pipeline Executes
   ├─ Deploy to Staging environment
   ├─ Run integration tests
   ├─ Manual approval gate
   └─ Deploy to Production

5. Monitor in Production
   ├─ Model Monitor tracks performance
   ├─ Data drift detection
   └─ Triggers retraining if needed
""")

In [ ]:
# ============================================================
# Example: Programmatic Model Approval
# ============================================================

print("Programmatic Model Approval Example")
print("=" * 60)

# This shows how to approve a model programmatically
# In practice, this would be done after reviewing metrics

approval_example = """
# Scenario: Pipeline trained a new fraud model
# Model registered as: fraud-detection-model-group version 3
# Status: PendingManualApproval

# Step 1: Review model metrics
model_package_arn = "arn:aws:sagemaker:region:account:model-package/fraud-detection-model-group/3"

# Retrieve model metrics from Model Registry
response = sm_client.describe_model_package(
    ModelPackageName=model_package_arn
)

metrics = response.get('ModelMetrics', {})
print("Model Performance:")
print(f"  AUC: {metrics.get('auc', 'N/A')}")
print(f"  Precision: {metrics.get('precision', 'N/A')}")
print(f"  Recall: {metrics.get('recall', 'N/A')}")

# Step 2: If metrics meet requirements, approve
# Decision criteria for fraud detection:
# - AUC >= 0.85 (good discrimination)
# - Precision >= 0.80 (low false positives)
# - Recall >= 0.75 (catch most fraud)

if meets_criteria:
    # Approve the model
    sm_client.update_model_package(
        ModelPackageArn=model_package_arn,
        ModelApprovalStatus='Approved',
        ApprovalDescription='Model meets production criteria for fraud detection'
    )
    print("✅ Model approved - Deployment pipeline will start automatically")
else:
    # Reject the model
    sm_client.update_model_package(
        ModelPackageArn=model_package_arn,
        ModelApprovalStatus='Rejected',
        ApprovalDescription='Model does not meet minimum performance thresholds'
    )
    print("❌ Model rejected - Pipeline needs retraining")
"""

print(approval_example)

print("\n\n🎯 Approval Criteria for Fraud Detection:")
print("-" * 60)
print("""
Metric                  Threshold    Rationale
────────────────────────────────────────────────────────
ROC-AUC                ≥ 0.85       Strong discrimination between fraud/legitimate
Precision              ≥ 0.80       Limit false positives (customer inconvenience)
Recall                 ≥ 0.75       Catch majority of fraudulent transactions
F1-Score               ≥ 0.77       Balanced performance
Data Drift Score       < 0.30       Model still relevant to current data
Latency (P95)          < 100ms      Real-time transaction approval
Model Size             < 500MB      Fast deployment and scaling

Compliance Checks:
✓ Model training data retention (audit trail)
✓ Feature importance explainability
✓ Bias testing across demographics
✓ Security scan (no vulnerabilities)
""")

### 🔍 Monitoring Project Status

Une fois le projet créé, vous pouvez surveiller son état et ses exécutions via la console SageMaker Studio ou programmatically.

In [ ]:
# ============================================================
# Monitor SageMaker Project Executions
# ============================================================

print("Monitoring Project Pipelines and Deployments")
print("=" * 60)

# Check project status
project_name = "fraud-detection-mlops"

try:
    # Get project details
    project = sm_client.describe_project(
        ProjectName=project_name
    )
    
    print(f"\n📁 Project: {project['ProjectName']}")
    print(f"   Status: {project['ProjectStatus']}")
    print(f"   Created: {project['CreationTime']}")
    print(f"   ID: {project['ProjectId']}")
    
except sm_client.exceptions.ResourceNotFound:
    print(f"\nProject '{project_name}' not found.")
    print("💡 Create project via SageMaker Studio → Projects → Create Project")
    
except Exception as e:
    print(f"\nDemo mode: {e}")

print("\n\n🔄 Recent Pipeline Executions:")
print("-" * 60)

# Show how to monitor pipeline executions
monitoring_example = """
# List recent pipeline executions
pipeline_name = "fraud-detection-pipeline"

executions = sm_client.list_pipeline_executions(
    PipelineName=pipeline_name,
    MaxResults=10,
    SortBy='CreationTime',
    SortOrder='Descending'
)

for execution in executions['PipelineExecutionSummaries']:
    status = execution['PipelineExecutionStatus']
    status_icon = "✅" if status == "Succeeded" else "🔄" if status == "Executing" else "❌"
    
    print(f"{status_icon} Execution: {execution['PipelineExecutionDisplayName']}")
    print(f"   Status: {status}")
    print(f"   Started: {execution['StartTime']}")
    
    # Get execution details
    details = sm_client.describe_pipeline_execution(
        PipelineExecutionArn=execution['PipelineExecutionArn']
    )
    
    # Show step status
    steps = sm_client.list_pipeline_execution_steps(
        PipelineExecutionArn=execution['PipelineExecutionArn']
    )
    
    print("   Steps:")
    for step in steps['PipelineExecutionSteps']:
        step_status = step['StepStatus']
        step_icon = "✅" if step_status == "Succeeded" else "🔄" if step_status == "Executing" else "❌"
        print(f"     {step_icon} {step['StepName']}: {step_status}")
"""

print(monitoring_example)

print("\n\n📊 Deployment Status:")
print("-" * 60)

deployment_monitoring = """
# Monitor CodePipeline deployment status
codepipeline = boto3.client('codepipeline')

pipeline_name = "sagemaker-fraud-detection-mlops-modeldeploy"

# Get pipeline state
response = codepipeline.get_pipeline_state(
    name=pipeline_name
)

print(f"Deployment Pipeline: {pipeline_name}")
print(f"Updated: {response['updated']}")
print()

for stage in response['stageStates']:
    print(f"Stage: {stage['stageName']}")
    
    for action in stage.get('actionStates', []):
        status = action.get('latestExecution', {}).get('status', 'N/A')
        status_icon = "✅" if status == "Succeeded" else "🔄" if status == "InProgress" else "❌"
        
        print(f"  {status_icon} {action['actionName']}: {status}")
        
        if action['actionName'] == 'DeployStaging':
            print(f"      → Staging endpoint: fraud-detection-staging")
        elif action['actionName'] == 'DeployProd':
            print(f"      → Production endpoint: fraud-detection-prod")
"""

print(deployment_monitoring)

---

## 📚 Résumé: SageMaker Projects vs Manual Deployment

**Comparaison des approches pour la détection de fraude:**

| Aspect | Manual Deployment | SageMaker Projects |
|--------|-------------------|-------------------|
| **Setup** | Configure chaque composant manuellement | Template pré-configuré avec CI/CD |
| **Code Management** | Notebooks locaux ou scripts ad-hoc | Repository Git avec versioning |
| **Training** | Lancer Training Jobs manuellement | Pipeline automatisé avec déclencheurs |
| **Quality Gates** | Vérification manuelle des métriques | Seuils automatiques + approbation |
| **Deployment** | Créer endpoints manuellement | Pipeline multi-environnements (Dev/Staging/Prod) |
| **Rollback** | Processus manuel complexe | Rollback automatique si échec |
| **Monitoring** | Configuration manuelle de Model Monitor | Intégré avec alertes CloudWatch |
| **Collaboration** | Partage de notebooks | Code review via pull requests |
| **Audit Trail** | Logs dispersés | Historique complet des déploiements |
| **Scalabilité** | Répétition manuelle pour nouveaux modèles | Réutilisation du template |

**Recommandation pour Production:**
- **POC/Experimentation**: Manual deployment (plus rapide pour tester)
- **Production**: SageMaker Projects (governance, automation, collaboration)

**Prochaine section**: Nous allons maintenant explorer les stratégies de déploiement (Blue/Green, Canary) en utilisant les modèles créés.

In [ ]:
# ============================================================
# Créer deux modèles (v1 et v2)
# ============================================================

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import joblib
import tarfile

# Générer des données
np.random.seed(42)
n_samples = 1000
X = np.random.randn(n_samples, 5)
y = (X[:, 0] + X[:, 1] > 0).astype(int)

# Modèle v1 - Simple (baseline)
model_v1 = LogisticRegression(random_state=42)
model_v1.fit(X, y)

# Modèle v2 - Plus complexe (improved)
model_v2 = RandomForestClassifier(n_estimators=50, max_depth=10, random_state=42)
model_v2.fit(X, y)

# Fonction pour sauvegarder et uploader
def save_and_upload_model(model, version):
    joblib.dump(model, 'model.pkl')
    
    with tarfile.open('model.tar.gz', 'w:gz') as tar:
        tar.add('model.pkl')
    
    model_s3_key = f'deployment-models/{version}/model.tar.gz'
    boto3.client('s3').upload_file('model.tar.gz', bucket, model_s3_key)
    model_s3_uri = f's3://{bucket}/{model_s3_key}'
    
    os.remove('model.pkl')
    os.remove('model.tar.gz')
    
    return model_s3_uri

model_v1_uri = save_and_upload_model(model_v1, 'v1')
model_v2_uri = save_and_upload_model(model_v2, 'v2')

print(f"Model v1 (Logistic): {model_v1_uri}")
print(f"Model v2 (RandomForest): {model_v2_uri}")

# Données de test
test_data = np.array([[0.5, 0.3, -0.2, 0.1, 0.4]])
print(f"\nTest data shape: {test_data.shape}")

# Comparer les prédictions
pred_v1 = model_v1.predict_proba(test_data)[0]
pred_v2 = model_v2.predict_proba(test_data)[0]

print(f"\nModel v1 proba: {pred_v1}")
print(f"Model v2 proba: {pred_v2}")

---

## Section 2: Blue/Green Deployment


In [ ]:
# ============================================================
# Étape 1: Déployer la version Blue (v1)
# ============================================================

# Créer le modèle v1 (Blue)
sklearn_model_v1 = SKLearnModel(
    model_data=model_v1_uri,
    role=role,
    framework_version='1.2-1',
    py_version='py3',
    name=f'model-v1-{int(time.time())}'
)

# Déployer Blue
endpoint_name_bg = f'blue-green-endpoint-{int(time.time())}'

predictor_blue = sklearn_model_v1.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.large',
    endpoint_name=endpoint_name_bg,
    serializer=CSVSerializer(),
    deserializer=CSVDeserializer()
)

print(f"Blue deployment (v1) complete")
print(f"Endpoint: {endpoint_name_bg}")

# Tester Blue
result_blue = predictor_blue.predict(test_data)
print(f"Blue prediction: {result_blue}")

In [ ]:
# ============================================================
# Étape 2: Déployer la version Green (v2) sur le même endpoint
# ============================================================

# Créer le modèle v2 (Green)
sklearn_model_v2 = SKLearnModel(
    model_data=model_v2_uri,
    role=role,
    framework_version='1.2-1',
    py_version='py3',
    name=f'model-v2-{int(time.time())}'
)

# Créer la configuration Green
production_variants = [
    {
        'VariantName': 'Blue',
        'ModelName': sklearn_model_v1.name,
        'InstanceType': 'ml.m5.large',
        'InitialInstanceCount': 1,
        'InitialVariantWeight': 1.0  # 100% du trafic
    },
    {
        'VariantName': 'Green',
        'ModelName': sklearn_model_v2.name,
        'InstanceType': 'ml.m5.large',
        'InitialInstanceCount': 1,
        'InitialVariantWeight': 0.0  # 0% du trafic (shadow)
    }
]

# Créer la nouvelle configuration
endpoint_config_name = f'bg-config-{int(time.time())}'

sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=production_variants
)

# Mettre à jour l'endpoint
sm_client.update_endpoint(
    EndpointName=endpoint_name_bg,
    EndpointConfigName=endpoint_config_name
)

print(f"\nGreen deployment (v2) started")
print(f"Endpoint config: {endpoint_config_name}")
print("Waiting for update to complete...")

# Attendre la mise à jour
waiter = sm_client.get_waiter('endpoint_in_service')
waiter.wait(EndpointName=endpoint_name_bg)

print("Green deployment complete (0% traffic)")

In [ ]:
# ============================================================
# Étape 3: Basculer le trafic de Blue vers Green
# ============================================================

# Tester d'abord avec TargetVariant
print("Testing Green variant before switching traffic...")

response = sm_runtime.invoke_endpoint(
    EndpointName=endpoint_name_bg,
    Body=','.join(map(str, test_data[0])),
    ContentType='text/csv',
    TargetVariant='Green'
)

result_green = response['Body'].read().decode()
print(f"Green prediction: {result_green}")

# Switch progressif: 50% Blue, 50% Green
print("\nSwitching to 50/50 traffic split...")

updated_variants = [
    {
        'VariantName': 'Blue',
        'DesiredWeight': 50.0
    },
    {
        'VariantName': 'Green',
        'DesiredWeight': 50.0
    }
]

sm_client.update_endpoint_weights_and_capacities(
    EndpointName=endpoint_name_bg,
    DesiredWeightsAndCapacities=updated_variants
)

print("Traffic split: 50% Blue, 50% Green")
time.sleep(5)

# Tester la répartition
print("\nTesting traffic distribution (10 requests):")
results = {'Blue': 0, 'Green': 0}

for i in range(10):
    response = sm_runtime.invoke_endpoint(
        EndpointName=endpoint_name_bg,
        Body=','.join(map(str, test_data[0])),
        ContentType='text/csv'
    )
    invoked_variant = response['ResponseMetadata']['HTTPHeaders'].get('x-amzn-invoked-production-variant', 'Unknown')
    results[invoked_variant] = results.get(invoked_variant, 0) + 1

print(f"Results: {results}")

In [ ]:
# ============================================================
# Étape 4: Basculer 100% vers Green
# ============================================================

print("\nSwitching to 100% Green (full cutover)...")

final_variants = [
    {
        'VariantName': 'Blue',
        'DesiredWeight': 0.0
    },
    {
        'VariantName': 'Green',
        'DesiredWeight': 100.0
    }
]

sm_client.update_endpoint_weights_and_capacities(
    EndpointName=endpoint_name_bg,
    DesiredWeightsAndCapacities=final_variants
)

print("Traffic: 100% Green")
time.sleep(5)

# Vérifier
print("\nTesting final distribution (10 requests):")
results_final = {'Blue': 0, 'Green': 0}

for i in range(10):
    response = sm_runtime.invoke_endpoint(
        EndpointName=endpoint_name_bg,
        Body=','.join(map(str, test_data[0])),
        ContentType='text/csv'
    )
    invoked_variant = response['ResponseMetadata']['HTTPHeaders'].get('x-amzn-invoked-production-variant', 'Unknown')
    results_final[invoked_variant] = results_final.get(invoked_variant, 0) + 1

print(f"Results: {results_final}")
print("\nBlue/Green deployment complete!")

---

## Section 3: Canary Deployment


In [ ]:
# ============================================================
# Canary Deployment: Déploiement progressif
# ============================================================

# Créer un nouvel endpoint pour Canary
endpoint_name_canary = f'canary-endpoint-{int(time.time())}'

# Configuration initiale: 90% Blue, 10% Canary
canary_variants = [
    {
        'VariantName': 'Production',
        'ModelName': sklearn_model_v1.name,
        'InstanceType': 'ml.m5.large',
        'InitialInstanceCount': 1,
        'InitialVariantWeight': 90.0
    },
    {
        'VariantName': 'Canary',
        'ModelName': sklearn_model_v2.name,
        'InstanceType': 'ml.m5.large',
        'InitialInstanceCount': 1,
        'InitialVariantWeight': 10.0
    }
]

canary_config_name = f'canary-config-{int(time.time())}'

sm_client.create_endpoint_config(
    EndpointConfigName=canary_config_name,
    ProductionVariants=canary_variants
)

sm_client.create_endpoint(
    EndpointName=endpoint_name_canary,
    EndpointConfigName=canary_config_name
)

print(f"Canary endpoint created: {endpoint_name_canary}")
print("Initial split: 90% Production, 10% Canary")
print("Waiting for endpoint to be in service...")

waiter = sm_client.get_waiter('endpoint_in_service')
waiter.wait(EndpointName=endpoint_name_canary)

print("Canary endpoint ready!")

In [ ]:
# ============================================================
# Tester la distribution Canary
# ============================================================

print("\nTesting Canary distribution (100 requests):")

canary_results = {'Production': 0, 'Canary': 0, 'Unknown': 0}

for i in range(100):
    try:
        response = sm_runtime.invoke_endpoint(
            EndpointName=endpoint_name_canary,
            Body=','.join(map(str, test_data[0])),
            ContentType='text/csv'
        )
        variant = response['ResponseMetadata']['HTTPHeaders'].get('x-amzn-invoked-production-variant', 'Unknown')
        canary_results[variant] = canary_results.get(variant, 0) + 1
    except Exception as e:
        print(f"Error: {e}")
        break

print(f"\nDistribution:")
print(f"Production: {canary_results['Production']}%")
print(f"Canary: {canary_results['Canary']}%")
print(f"Unknown: {canary_results.get('Unknown', 0)}%")

In [ ]:
# ============================================================
# Progression Canary: 10% → 25% → 50% → 100%
# ============================================================

canary_stages = [
    {'Production': 75.0, 'Canary': 25.0},
    {'Production': 50.0, 'Canary': 50.0},
    {'Production': 0.0, 'Canary': 100.0}
]

for i, stage in enumerate(canary_stages, start=2):
    print(f"\nCanary Stage {i}: Production {stage['Production']}%, Canary {stage['Canary']}%")
    
    updated = [
        {'VariantName': 'Production', 'DesiredWeight': stage['Production']},
        {'VariantName': 'Canary', 'DesiredWeight': stage['Canary']}
    ]
    
    sm_client.update_endpoint_weights_and_capacities(
        EndpointName=endpoint_name_canary,
        DesiredWeightsAndCapacities=updated
    )
    
    print("Waiting 10 seconds for monitoring...")
    time.sleep(10)
    
    # Tester la distribution
    test_results = {'Production': 0, 'Canary': 0}
    for j in range(20):
        response = sm_runtime.invoke_endpoint(
            EndpointName=endpoint_name_canary,
            Body=','.join(map(str, test_data[0])),
            ContentType='text/csv'
        )
        variant = response['ResponseMetadata']['HTTPHeaders'].get('x-amzn-invoked-production-variant', 'Unknown')
        test_results[variant] = test_results.get(variant, 0) + 1
    
    print(f"Verified: Production {test_results['Production']}, Canary {test_results['Canary']}")

print("\n✅ Canary deployment complete - 100% on new version")

---

## Section 4: Monitoring et Métriques CloudWatch


In [ ]:
# ============================================================
# Récupérer les métriques CloudWatch
# ============================================================

def get_endpoint_metrics(endpoint_name, variant_name, metric_name, minutes=10):
    """Récupère les métriques CloudWatch pour un endpoint"""
    
    end_time = datetime.utcnow()
    start_time = end_time - timedelta(minutes=minutes)
    
    response = cloudwatch.get_metric_statistics(
        Namespace='AWS/SageMaker',
        MetricName=metric_name,
        Dimensions=[
            {'Name': 'EndpointName', 'Value': endpoint_name},
            {'Name': 'VariantName', 'Value': variant_name}
        ],
        StartTime=start_time,
        EndTime=end_time,
        Period=60,
        Statistics=['Average', 'Sum']
    )
    
    return response['Datapoints']

# Métriques pour Blue/Green endpoint
print("CloudWatch Metrics pour Blue/Green Endpoint:\n")

metrics_to_check = ['Invocations', 'ModelLatency', 'Invocation4XXErrors', 'Invocation5XXErrors']

for variant in ['Blue', 'Green']:
    print(f"\n{variant} Variant:")
    for metric in metrics_to_check:
        datapoints = get_endpoint_metrics(endpoint_name_bg, variant, metric, minutes=15)
        if datapoints:
            latest = sorted(datapoints, key=lambda x: x['Timestamp'])[-1]
            value = latest.get('Sum', latest.get('Average', 0))
            print(f"  {metric}: {value:.2f}")
        else:
            print(f"  {metric}: No data")

# Métriques pour Canary endpoint
print("\n\nCloudWatch Metrics pour Canary Endpoint:\n")

for variant in ['Production', 'Canary']:
    print(f"\n{variant} Variant:")
    for metric in metrics_to_check:
        datapoints = get_endpoint_metrics(endpoint_name_canary, variant, metric, minutes=15)
        if datapoints:
            latest = sorted(datapoints, key=lambda x: x['Timestamp'])[-1]
            value = latest.get('Sum', latest.get('Average', 0))
            print(f"  {metric}: {value:.2f}")
        else:
            print(f"  {metric}: No data")

In [ ]:
# ============================================================
# Créer une alarme CloudWatch
# ============================================================

alarm_name = f'canary-error-rate-{int(time.time())}'

# Alarme sur le taux d'erreur 4XX/5XX
cloudwatch.put_metric_alarm(
    AlarmName=alarm_name,
    ComparisonOperator='GreaterThanThreshold',
    EvaluationPeriods=2,
    MetricName='Invocation4XXErrors',
    Namespace='AWS/SageMaker',
    Period=60,
    Statistic='Sum',
    Threshold=10.0,
    ActionsEnabled=False,  # Désactivé pour le lab
    AlarmDescription='Alert when Canary has high error rate',
    Dimensions=[
        {'Name': 'EndpointName', 'Value': endpoint_name_canary},
        {'Name': 'VariantName', 'Value': 'Canary'}
    ]
)

print(f"CloudWatch Alarm créée: {alarm_name}")
print("Cette alarme se déclencherait si >10 erreurs 4XX en 2 minutes")
print("\nEn production, l'alarme pourrait déclencher:")
print("- Rollback automatique")
print("- Notification SNS")
print("- Lambda pour corriger")

---

## Section 5: Rollback et A/B Testing


In [ ]:
# ============================================================
# Rollback: Revenir à la version précédente
# ============================================================

print("Simulation d'un problème détecté sur Green...")
print("Initiating ROLLBACK...\n")

# Rollback = remettre 100% sur Blue
rollback_variants = [
    {
        'VariantName': 'Blue',
        'DesiredWeight': 100.0
    },
    {
        'VariantName': 'Green',
        'DesiredWeight': 0.0
    }
]

sm_client.update_endpoint_weights_and_capacities(
    EndpointName=endpoint_name_bg,
    DesiredWeightsAndCapacities=rollback_variants
)

print("✅ Rollback complete - 100% sur Blue (version stable)")
time.sleep(5)

# Vérifier
print("\nTesting après rollback (10 requests):")
rollback_results = {'Blue': 0, 'Green': 0}

for i in range(10):
    response = sm_runtime.invoke_endpoint(
        EndpointName=endpoint_name_bg,
        Body=','.join(map(str, test_data[0])),
        ContentType='text/csv'
    )
    variant = response['ResponseMetadata']['HTTPHeaders'].get('x-amzn-invoked-production-variant', 'Unknown')
    rollback_results[variant] = rollback_results.get(variant, 0) + 1

print(f"Results: {rollback_results}")

In [ ]:
# ============================================================
# A/B Testing avec analyse statistique
# ============================================================

print("\nA/B Testing: Comparer les performances de 2 variants\n")

# Configuration 50/50 pour A/B test
ab_variants = [
    {
        'VariantName': 'Blue',
        'DesiredWeight': 50.0
    },
    {
        'VariantName': 'Green',
        'DesiredWeight': 50.0
    }
]

sm_client.update_endpoint_weights_and_capacities(
    EndpointName=endpoint_name_bg,
    DesiredWeightsAndCapacities=ab_variants
)

print("A/B Test configuration: 50% Blue, 50% Green")
time.sleep(5)

# Collecter les métriques
ab_results = {
    'Blue': {'latencies': [], 'predictions': []},
    'Green': {'latencies': [], 'predictions': []}
}

print("\nRunning A/B test (50 requests)...")
for i in range(50):
    start = time.time()
    response = sm_runtime.invoke_endpoint(
        EndpointName=endpoint_name_bg,
        Body=','.join(map(str, test_data[0])),
        ContentType='text/csv'
    )
    latency = (time.time() - start) * 1000
    
    variant = response['ResponseMetadata']['HTTPHeaders'].get('x-amzn-invoked-production-variant', 'Unknown')
    prediction = response['Body'].read().decode()
    
    if variant in ab_results:
        ab_results[variant]['latencies'].append(latency)
        ab_results[variant]['predictions'].append(prediction)

# Analyser les résultats
print("\nA/B Test Results:\n")
for variant, data in ab_results.items():
    if data['latencies']:
        avg_latency = np.mean(data['latencies'])
        p50_latency = np.percentile(data['latencies'], 50)
        p95_latency = np.percentile(data['latencies'], 95)
        
        print(f"{variant}:")
        print(f"  Requests: {len(data['latencies'])}")
        print(f"  Avg Latency: {avg_latency:.2f}ms")
        print(f"  P50 Latency: {p50_latency:.2f}ms")
        print(f"  P95 Latency: {p95_latency:.2f}ms")
        print()

print("Décision A/B Testing:")
if len(ab_results['Green']['latencies']) > 0 and len(ab_results['Blue']['latencies']) > 0:
    green_avg = np.mean(ab_results['Green']['latencies'])
    blue_avg = np.mean(ab_results['Blue']['latencies'])
    
    if green_avg < blue_avg * 1.1:  # Green pas plus de 10% plus lent
        print("✅ Green wins - Better or similar performance")
    else:
        print("❌ Blue wins - Green is significantly slower")

---

## Section 6: Cleanup


In [ ]:
# ============================================================
# Cleanup
# ============================================================

print("Cleaning up resources...\n")

# Supprimer les endpoints
endpoints_to_delete = [endpoint_name_bg, endpoint_name_canary]

for ep_name in endpoints_to_delete:
    try:
        sm_client.delete_endpoint(EndpointName=ep_name)
        print(f"✅ Deleted endpoint: {ep_name}")
    except Exception as e:
        print(f"❌ Could not delete {ep_name}: {e}")

# Supprimer l'alarme CloudWatch
try:
    cloudwatch.delete_alarms(AlarmNames=[alarm_name])
    print(f"✅ Deleted alarm: {alarm_name}")
except Exception as e:
    print(f"❌ Could not delete alarm: {e}")

print("\nCleanup complete!")

---

## Résumé

Dans ce lab, vous avez:

1. **Implémenté Blue/Green Deployment**
   - Déployé deux versions simultanément
   - Basculé progressivement le trafic (0% → 50% → 100%)
   - Effectué un rollback rapide

2. **Configuré Canary Deployment**
   - Déploiement progressif (10% → 25% → 50% → 100%)
   - Monitoring à chaque étape
   - Validation avant progression

3. **Monitoré avec CloudWatch**
   - Métriques de latence et d'erreurs
   - Alarmes pour détection automatique
   - Analyse comparative des variants

4. **Testé A/B Testing**
   - Comparaison statistique de performances
   - Décision basée sur les données
   - Distribution 50/50 du trafic

5. **Géré les rollbacks**
   - Retour rapide à la version stable
   - Sans downtime
   - Automatisable via alarmes

### Comparaison des Stratégies

| Stratégie | Risque | Vitesse | Rollback | Cas d'Usage |
|-----------|--------|---------|----------|-------------|
| **Blue/Green** | Moyen | Rapide | Instantané | Mises à jour majeures |
| **Canary** | Faible | Progressive | Facile | Production critique |
| **A/B Testing** | Faible | Lente | N/A | Optimisation |
| **Shadow** | Très faible | Variable | N/A | Validation |

### Best Practices

1. **Blue/Green**: Idéal pour basculement rapide avec rollback instantané
2. **Canary**: Préféré en production pour minimiser les risques
3. **Monitoring**: Toujours surveiller les métriques (latence, erreurs, coût)
4. **Alarmes**: Configurer des seuils pour rollback automatique
5. **Tests**: Valider chaque variant avant d'augmenter le trafic

### Métriques Clés à Surveiller

- **Latency**: P50, P95, P99
- **Error Rate**: 4XX, 5XX
- **Throughput**: Invocations/minute
- **Model Performance**: Accuracy, drift
- **Cost**: Instance utilization

### Next Steps

- Implémenter Lambda pour rollback automatique
- Intégrer avec CI/CD (CodePipeline)
- Ajouter Model Monitor pour data drift
- Configurer SNS pour notifications
- Automatiser les tests de charge

---

**Best Practice**: Toujours utiliser Canary pour les déploiements en production critique
